In [29]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph
from autocnet.graph.edge import Edge
from autocnet.matcher.matcher import FlannMatcher

from scipy.misc import imresize
import math

from IPython.display import display

%matplotlib inline

# Create Basic Structures

In [30]:
#Point to the adjacency Graph
adjacency = get_path('three_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':500})

#Match
cg.match_features()

# Perform the symmetry check
cg.symmetry_checks()
# Perform the ratio check
cg.ratio_checks(clean_keys = ['symmetry'])
# Create fundamental matrix
cg.compute_fundamental_matrices(clean_keys = ['symmetry', 'ratio'])


# Step: Compute the homographies and apply RANSAC
cg.compute_homographies(clean_keys=['symmetry', 'ratio'])

# Step: Compute the overlap ratio and coverage ratio
for s, d, edge in cg.edges_iter(data=True):
    edge.coverage_ratio(clean_keys=['symmetry', 'ratio'])

# Step: Compute subpixel offsets for candidate points
cg.subpixel_register(clean_keys=['ransac'])

cg.suppress()

/scratch/autocnet/autocnet/matcher/outlier_detector.py:261: UserWarning: Unable to optimally solve.  Returning with 276 points
  warnings.warn('Unable to optimally solve.  Returning with {} points'.format(len(result)))


# Def all the things

In [31]:
def cifi(arr, scales=[0.5, 0.57, 0.66,  0.76, 0.87, 1.0], radii=list(range(1,12))):
    
    result = np.empty((len(scales), len(radii)))
    
    for i, s in enumerate(scales):
        new_arr = imresize(arr, s)
        for j, r in enumerate(radii):
            inv_area = 1 / (2 * math.pi * r)
            r *= r
            # Generate a circular mask
            a, b = (int(new_arr.shape[0] / 2),
                    int(new_arr.shape[1] / 2))
            
            nx, ny = new_arr.shape
            y, x = np.ogrid[-a: ny - a, -b: nx - b]
            mask = x*x + y* y <= r**2

            # Reset the center pixels to be False
            y, x = np.ogrid[-a: ny - a, -b: nx - b]
            mask2 = x*x + y* y <= (r - 1)**2
            mask[mask2==True] = False
            
            s = np.sum(new_arr[mask]) * inv_area
            
            if s == 0:
                s = -1
            result[i,j] = s
    return result
    
def cifi2(arr, radii=list(range(1,12))):
    result = np.empty((arr.shape[0], arr.shape[1], len(radii)))
    
    for i, y in enumerate(range(arr.shape[0])):
        for j, x in enumerate(range(arr.shape[1])):
            for k, r in enumerate(radii):
                inv_area = 1 / (2 * math.pi * r)
                r *= r
                # Generate a circular mask
                a, b = i, j

                nx, ny = arr.shape
                y, x = np.ogrid[-a: ny - a, -b: nx - b]
                mask = x*x + y* y <= r**2

                # Reset the center pixels to be False
                y, x = np.ogrid[-a: ny - a, -b: nx - b]
                mask2 = x*x + y* y <= (r - 1)**2
                mask[mask2==True] = False

                s = np.sum(arr[mask]) * inv_area

                if s == 0:
                    s = -1
                result[i, j, k] = s
    return result
    
def ciratefi(template, image, upsampling=1):
    if upsampling < 1:
        raise ValueError

    u_template = zoom(template, upsampling, order=1)
    u_image = zoom(image, upsampling, order=1)
    
    cifi_template = cifi(u_template)
    cifi_search = cifi2(u_image)
    
    coeffs = np.empty((cifi_search.shape[0], cifi_search.shape[1]))
    for y, l in enumerate(range(cifi_search.shape[0])):
        for x, m in enumerate(range(cifi_search.shape[1])):
            for i, k in enumerate(range(cifi_template.shape[0])):
                maxcoeff = -math.inf
                coeff = np.corrcoef(cifi_template[i], cifi_search[y,x])[0,1]
                if coeff > maxcoeff:
                    maxcoeff = coeff
            coeffs[y, x] = maxcoeff
    imshow(coeffs, interpolation='none')
    colorbar()

# Do Stuff

In [32]:
from scipy.ndimage.interpolation import zoom
from scipy.stats.stats import pearsonr 

figsize(10,10)
e = cg.edge[0][1]
matches = e.matches
clean_keys = ['suppression']

full_offsets = np.zeros((len(matches), 3))

if clean_keys:
    matches, mask = e._clean(clean_keys)

# Preallocate the numpy array to avoid appending and type conversion
edge_offsets = np.empty((len(matches),3))

# for each edge, calculate this for each keypoint pair
for i, (idx, row) in enumerate(matches.iterrows()):
    s_idx = int(row['source_idx'])
    d_idx = int(row['destination_idx'])
    s_kps = e.source.get_keypoints().iloc[s_idx]
    d_kps = e.destination.get_keypoints().iloc[d_idx]

    s_keypoint = e.source.get_keypoints().iloc[s_idx][['x', 'y']].values
    d_keypoint = e.destination.get_keypoints().iloc[d_idx][['x', 'y']].values

    # Get the template and search windows
    s_template = sp.clip_roi(e.source.handle, s_keypoint,9)
    d_search = sp.clip_roi(e.destination.handle, d_keypoint, 51)
    imshow(s_template, cmap='Greys')
    show()
    imshow(d_search, cmap='Greys')
    show()
    
    ciratefi(s_template, d_search, upsampling=1)
    break

NameError: name 'sp' is not defined